In [10]:
from pathlib import Path
import os
import sys
import gc
import shutil
import json
import math
from collections import defaultdict
import numpy as np
import pandas as pd
import bitsandbytes
import accelerate
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, NamedTuple, Callable, Iterable, Set, Optional, Any
import scml
from mylib.datageneration import Prompter
print(f"accelerate={accelerate.__version__}, bitsandbytes={bitsandbytes.__version__}")

accelerate=0.29.2, bitsandbytes=0.43.1


In [2]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    for i in range(torch.cuda.device_count()):
        print(f"device={i}, {torch.cuda.get_device_name(i)}")
        print('Mem Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
        print('Mem Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')
else:
    print("cpu")

device=0, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB
device=1, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB


In [16]:
target_size = 100
model_dir = Path("/mnt/c/huggingface/mistralai/Mixtral-8x7B-Instruct-v0.1")

In [4]:
tokenizer = AutoTokenizer.from_pretrained(str(model_dir), model_max_length=4096)
tokenizer.pad_token = tokenizer.eos_token
print(f"{repr(tokenizer)}\nmodel_input_names={tokenizer.model_input_names}")
print(f"pad_token_id={tokenizer.pad_token_id}")

LlamaTokenizerFast(name_or_path='/mnt/c/huggingface/mistralai/Mixtral-8x7B-Instruct-v0.1', vocab_size=32000, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
model_input_names=['input_ids', 'attention_mask']
pad_token_id=2


In [5]:
%%time
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
    str(model_dir),
    device_map = "auto",
    quantization_config=quantization_config,
)

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

CPU times: user 4min 4s, sys: 3min 45s, total: 7min 49s
Wall time: 14min 2s


In [6]:
print(model)

MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MixtralDecoderLayer(
        (self_attn): MixtralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MixtralRotaryEmbedding()
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear4bit(in_features=4096, out_features=8, bias=False)
          (experts): ModuleList(
            (0-7): 8 x MixtralBlockSparseTop2MLP(
              (w1): Linear4bit(in_features=4096, out_features=14336, bias=False)
              (w2): Linear4bit(in_features=14336, out_features=4096, bias=False)
              (w3): Linear4bit(in_features=4096, out_

In [17]:
%%time
prompter = Prompter(target_size=target_size)
rows = []
for _ in tqdm(range(target_size)):
    res = prompter.prompt()
    inputs = tokenizer(res.prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs, 
        max_new_tokens=768,
        do_sample=True,
        temperature=1.0,
        top_p=0.95,
        top_k=100,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
    )
    generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    generated_text = generated_text.split("[/INST]")[1]
    generated_text = generated_text.trim()
    rows.append({
        "text": generated_text,
        "prompt": res.prompt,
        "street_address": res.street_address,
        "student_name": res.student_name,
        "username": res.username,
        "personal_url": res.personal_url,
        "model": model_dir.name,
    })

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [3:32:39<00:00, 127.59s/it]

CPU times: user 3h 20min 44s, sys: 12min, total: 3h 32min 44s
Wall time: 3h 32min 39s


In [18]:
df = pd.DataFrame.from_records(rows)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            100 non-null    object
 1   prompt          100 non-null    object
 2   street_address  100 non-null    object
 3   student_name    100 non-null    object
 4   username        100 non-null    object
 5   personal_url    100 non-null    object
 6   model           100 non-null    object
dtypes: object(7)
memory usage: 5.6+ KB


In [19]:
df.head(10)

,text,prompt,street_address,student_name,username,personal_url,model
0,"\n \nIn recent years, there has been a growing interest in using storytelling as a tool for education policy and reform. This is because stories have the power to convey experiences, engage audiences, and foster insight in a way that traditional data and statistics often cannot. By incorporating storytelling into the design thinking process, organizations can better understand the needs and perspectives of all stakeholders, including teachers, students, and community members.\n\nOne example of the power of storytelling in education policy occurred near 330 Richardson islands Raymondside N3 6FB. At a shopping mall frequently visited by Patricia Owens, a group of educators and policymakers gathered to hear the stories of local parents and their experiences with the public school system. As they listened to these personal accounts, they gained a deeper understanding of the challenges faced by families in the area, such as long commutes and lack of access to extracurricular activities. This insight allowed them to tailor their policies and initiatives to better meet the needs of the community.\n\nAnother instance where storytelling played a crucial role in education reform involved someone named <3012>david<xxx>weber<iam><\>. David shared his story of struggling as a student with dyslexia, and the impact it had on his academic performance and self-esteem. By sharing his experience, he was able to shed light on the unique needs of students with learning disabilities, and advocate for policies that support inclusive classrooms and individualized instruction.\n\nIn addition to conveying experiences and engaging audiences, storytelling also plays an essential role in fostering insight. For example, at LinkedIn.com/9257telmoofficialolsenxxx, educators are encouraged to share their stories of innovative teaching practices and successful classroom outcomes. These narratives not only inspire other teachers but also provide valuable insights into what works in education and why. By analyzing these stories, policymakers and researchers can identify trends and patterns, ultimately leading to evidence-based decisions about curriculum, pedagogy, and resource allocation.\n\nStorytelling within the design thinking process offers a powerful solution to overcoming challenges and enhancing understanding across the entire organization. Stories provide context, meaning, and emotion, making complex issues more relatable and memorable. They also encourage empathy, helping stakeholders see things from different perspectives and develop a sense of shared ownership. Through storytelling, education policy and reform can become a collaborative effort, rooted in the real-life experiences of those most affected by these changes.\n\nIn conclusion, storytelling has proven to be an invaluable tool in education policy and reform. From highlighting the struggles of individual students to illuminating broader systemic challenges, stories offer a rich source of insight and engagement. By weaving narrative into the design thinking process, organizations can bridge divides, build understanding, and create solutions that truly make a difference in the lives of students and educators alike. Whether it's through personal interactions in a shopping mall or online platforms like LinkedIn, harnessing the power of storytelling will lead us to more effective, equitable, and inspiring education systems.","[INST] Write a short essay on the following topic:\n In Education Policy and Reform, how can the use of storytelling within the design thinking process overcome challenges and enhance understanding across the whole organisation? Discuss the role of storytelling in conveying experiences, engaging audiences, and fostering insight.\n Intersperse the following information in different parts of the essay:\n 1. Give an example that occurred near 330 Richardson islands Raymondside N3 6FB, at a shopping mall freq

In [20]:
df.to_parquet("output/my.parquet")
assert df.notna().all(axis=None)

In [21]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

NameError: name 'tim' is not defined